In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

# Read the dataframe
df = pd.read_pickle('augmented_data.pkl')
df

,id,embedding,medoids,cluster_sizes
0,1,"[[-0.0019472323, 0.058371827, 0.0812831, 0.030...","[[-0.004593551, 0.051833656, -0.013445671, -0....","[10, 7, 9, 1]"
1,2,"[[0.01151042, -0.021297293, -0.004139077, 0.03...","[[0.004454516, 0.011180584, 0.053998474, -0.02...","[6, 2, 5, 1]"
2,6,"[[0.013927452, 0.035443924, 0.016817052, -0.01...","[[-0.0025131523, 0.072745346, 0.04038468, -0.0...","[2, 2, 2, 4]"
3,9,"[[0.02544553, -0.03236037, 0.0035475865, 0.070...","[[-0.013504671, 0.07948076, 0.097698964, 0.042...","[2, 2, 1, 1]"
4,13,"[[0.017213065, -0.013364788, 0.013486441, -0.0...","[[-0.0059717577, 0.035555597, 0.024298443, -0....","[3, 3, 1, 3]"
...,...,...,...,...
15742,909992,"[[-0.07872735, -0.009727927, 0.023001013, -0.0...","[[0.020977847, 0.00994313, 0.016100913, -0.020...","[3, 1, 1, 1]"
15743,910046,"[[0.0024761495, 0.029174268, -0.121854655, 0.0...","[[-0.04889003, -0.027657501, -0.03703226, 0.00...","[3, 1, 2, 1]"
15744,910075,"[[-0.03798147, 0.0035953722, 0.03408878, 0.035...","[[0.011452884, 0.14479369, -0.02908832, 0.0719...","[3, 2, 1, 1]"
15745,910092,"[[-0.022506248, -0.034485348, -0.053791, 0.072...","[[-0.022506248, -0.034485348, -0.053791, 0.072...","[2, 1, 2, 1]"


In [2]:

# Convert medoids to list of numpy arrays
medoids_list = df['medoids'].apply(lambda x: np.array(x))#.reshape(-1))

# Stack them into a single numpy array and convert to PyTorch tensor
medoids_np = np.stack(medoids_list.to_numpy())
print(medoids_np[0].shape)
medoids_tensor = torch.FloatTensor(medoids_np)

# Create DataLoader
dataset = TensorDataset(medoids_tensor)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")





(4, 384)
Using cuda device


In [3]:
# from model import SimpleAutoencoder
# # Model Initialization
# model = SimpleAutoencoder().to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.MSELoss()
# print(next(iter(dataloader))[0].shape)

# from model import CNN_Autoencoder
# # Model Initialization
# model = CNN_Autoencoder().to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.MSELoss()
# print(next(iter(dataloader))[0].shape)


# from model import RecurrentAutoencoder
# # Load the trained model
# model = RecurrentAutoencoder()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.MSELoss()
# print(next(iter(dataloader))[0].shape)

from model import TransformerAutoencoder
# Example usage
embed_dim = 384  # Example embedding dimension
num_heads = 4    # Example number of heads in multi-head attention
dim_feedforward = 1024  # Example feedforward dimension
num_layers = 2  # Example number of layers in the transformer encoder
seq_length = 4  # Original sequence length

model = TransformerAutoencoder(embed_dim, num_heads, dim_feedforward, num_layers, seq_length).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
print(next(iter(dataloader))[0].shape)

torch.Size([256, 4, 384])


In [4]:

# Training Loop
num_epochs = 200
best_loss = float('inf')  # Initialize with a very high value

for epoch in range(num_epochs):
    for batch_idx, (data,) in enumerate(dataloader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()

        # Check if this is the best model so far
        if loss.item() < best_loss:
            best_loss = loss.item()
            # Save the model state
            torch.save(model.state_dict(), 'trained_TransformerAutoencoder_best.pth')

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/200], Loss: 0.0020
Epoch [2/200], Loss: 0.0018
Epoch [3/200], Loss: 0.0017
Epoch [4/200], Loss: 0.0016
Epoch [5/200], Loss: 0.0015
Epoch [6/200], Loss: 0.0015
Epoch [7/200], Loss: 0.0015
Epoch [8/200], Loss: 0.0014
Epoch [9/200], Loss: 0.0014
Epoch [10/200], Loss: 0.0014
Epoch [11/200], Loss: 0.0014
Epoch [12/200], Loss: 0.0014
Epoch [13/200], Loss: 0.0014
Epoch [14/200], Loss: 0.0013
Epoch [15/200], Loss: 0.0013
Epoch [16/200], Loss: 0.0014
Epoch [17/200], Loss: 0.0014
Epoch [18/200], Loss: 0.0014
Epoch [19/200], Loss: 0.0013
Epoch [20/200], Loss: 0.0013
Epoch [21/200], Loss: 0.0013
Epoch [22/200], Loss: 0.0014
Epoch [23/200], Loss: 0.0013
Epoch [24/200], Loss: 0.0014
Epoch [25/200], Loss: 0.0014
Epoch [26/200], Loss: 0.0013
Epoch [27/200], Loss: 0.0013
Epoch [28/200], Loss: 0.0013
Epoch [29/200], Loss: 0.0014
Epoch [30/200], Loss: 0.0013
Epoch [31/200], Loss: 0.0013
Epoch [32/200], Loss: 0.0013
Epoch [33/200], Loss: 0.0013
Epoch [34/200], Loss: 0.0013
Epoch [35/200], Loss: 0

In [5]:
# def encode_dataConv(model, numpy_arrays):
#     encoded_tensors = []  # To collect the encoded tensors
    
#     for arr in numpy_arrays:
#         # Check if the array can be reshaped to (1, 4, 384)
#         if arr.size == 1536:
#             reshaped_array = arr.reshape(1, 4, 384)
            
#             # Convert NumPy array to PyTorch tensor
#             input_tensor = torch.tensor(reshaped_array, dtype=torch.float32).to(device)
            
#             with torch.no_grad():  # Disable gradient computation
#                 encoded_tensor = model.encoder(input_tensor)  # Use only the encoder part
#                 encoded_tensors.append(encoded_tensor.cpu())  # Move tensor back to CPU
#         else:
#             print(f"Skipping array of shape {arr.shape}. Cannot be reshaped to (1, 4, 384).")
    
#     return encoded_tensors  # Return the list of all encoded tensors
# random_array = np.random.randn(256, 4, 384)

# encoded_tensors = encode_dataConv(model, random_array)
# print(len(encoded_tensors))
# encoded_tensors[0].shape